In [1]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dictances

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import division
import json
import numpy as np
from decimal import Decimal
from numpy import expand_dims, array, exp, max
import scipy
from scipy.special import rel_entr
import csv
import statistics
from numpy import expand_dims, array, exp, max
import math
from scipy.spatial import distance
from scipy.stats import wasserstein_distance
import numpy as np
from numpy.linalg import norm
import sklearn
from tensorflow import keras
import string
from scipy.special import rel_entr
from tensorflow.keras.applications.resnet50 import ResNet50 as resnet50
from tensorflow.keras.applications.vgg16 import VGG16 as vgg16
from tensorflow.keras.applications.densenet import DenseNet169 as densenet
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inceptionv3
from tensorflow.keras.applications.efficientnet import EfficientNetB1 as efficient
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mobilenetv2
from tensorflow.keras.applications.mobilenet import MobileNet as mobilenet
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
import csv
# import the timeit module
import timeit
import tracemalloc

In [4]:
from numpy.linalg import norm
from dictances import bhattacharyya, bhattacharyya_coefficient

In [5]:
# import the timeit module
import timeit
import math
import tracemalloc

In [6]:
def hellinger_fast(p, q):
    """Hellinger distance between two discrete distributions.
       In pure Python.
       Fastest version.
    """
    return sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p,q) ])

In [7]:
# to suppress division errors
np.seterr(divide='ignore', invalid='ignore')
new_final_source_array = []

In [33]:
# Opening final JSON files
contrast_source_data = open('../paper3/files/glcm/source_domain_data_new_energy_glcm.json')

#for target data
contrast_target_data = open('files/Caltech256/DenseNet169/glcm/caltech_domain_data_new_energy_glcm.json')

In [34]:
# returns JSON object
contrast_source_data_d = json.load(contrast_source_data)

contrast_target_data_d = json.load(contrast_target_data)

In [35]:
# kdl dictionary
kdl_dictionary = {}

In [36]:
def KL(p, q):
    a = []
    b = []
    for k in p:
        a.append(Decimal(k))

    for l in q:
        b.append(Decimal(l))

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    return np.sum(np.where(((a != 0) & (b != 0)), a * np.log(a / b), 0))

In [37]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [38]:
# get the data for the contrast target image
contrast_target_data_array = []
for p in range(0, len(contrast_target_data_d['target_images'])):
    for i, j in contrast_target_data_d['target_images'][p].items():
        contrast_target_data_array.append((i,np.array(j)))
contrast_target_data.close()

In [39]:
print(len(contrast_target_data_array))

7710


In [40]:
###contrast comparison code
contrast_All_Images = []
contrast_average_dkl = []

#check the timer_start
start_hellinger = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

for x in contrast_target_data_array[:1]:
#for x in target_data_array:
    t_images_btween_0_05 = []
    t_images_btween_05_1 = []
    t_images_btween_1_2 = []
    t_images_grt_2_5 = []
    t_images_grt_5_10 = []
    t_images_grt_10 = []
    
    image_name_class =  x[0].split('_/') #name[0]; class[1]
    compare_min_max = []
    
    
            
    # get the data for the source images
    for m in range(0, len(contrast_source_data_d['source_images'])):
        #print("source",source_data_d['source_images'][m][0])
        for k, v in contrast_source_data_d['source_images'][m].items():
            # compare the value of v with that of the target image
            # if 'caterpillar' in k.lower():
            # remove the zeros from the list
            new_v = np.array(v).flatten()
            #get the top items in x
            items = np.sort(x[1])
            item_size = items[-len(new_v):]
            new_x = item_size.flatten()
            
            new_d = softMax(0.5, new_x)     
            new_y = softMax(0.5, new_v)
            
            kdl = rel_entr( new_x, new_v, out=None)
            new_bhattacharyya = bhattacharyya(dict(enumerate(new_x.flatten(), 1)),dict(enumerate(new_v.flatten(), 1)) )
            new_wasserstein = wasserstein_distance(new_x,new_v)
            new_jensen = distance.jensenshannon(new_x,new_v)
            new_hellinger = hellinger_fast(new_x,new_v)
            
            compare_min_max.append((image_name_class[0],image_name_class[1],k, max(new_bhattacharyya)))
            
contrast_All_Images.append(compare_min_max)
    
contrast_source_data.close()

stop = timeit.default_timer()
execution_time = stop - start_hellinger
print("KDL -selected points execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()


model_name = densenet
input_t = keras.Input(shape=(224, 224, 3))
model = model_name(include_top=False,
                   weights="imagenet",
                   input_tensor=input_t)


# get the layer index
def getLayerIndex(model_i, layer_name):
    for pos, layer_g in enumerate(model_i.layers):
        if layer_g.name == layer_name:
            return pos


# get the convolved layers into an array for looping
convolved_layers = []
convolved_layer_names = []
# create a list to place them
layer_cosine_list = []
layer_final_cos_list = []

for layer in model.layers[0:-2]:
    if 'conv' not in layer.name:
        continue

    index = getLayerIndex(model, layer.name)
    #print(layer.name)
    # append the convolved layer
    convolved_layers.append(index)
    convolved_layer_names.append(layer)

jdk = {}
for lyr in convolved_layer_names:
    ary = np.array(lyr.get_weights(), dtype=object)
    layer_index = getLayerIndex(model, lyr.name)
    if len(ary) != 0:
        # filters, biases = layer.get_weights()
        # print(filters.shape)
        ary = np.array(lyr.get_weights(), dtype=object)

        zipper_dict = {}
        dict_array = []

        # check for the arrays
        for x in ary:
            # print(len(x))
            # find if the array is 1 dim
            if x.ndim > 1:
                for y in x:
                    for z in y:
                        u = 0
                        for za in z:
                            # get all the positive elements in this and push them into an array
                            positive_elements = []
                            for pos_item in za:
                                if pos_item > 0:
                                    positive_elements.append(pos_item)
                            u = u + 1
                            zipper_dict.update({u: positive_elements})

            elif x.ndim == 1:
                # get the items/arrays with more than the 1 dimension
                print("")
        # print("lyr:",lyr.name)
        # get two items per tuple
        for i in range(1, len(zipper_dict), 2):
            if len(zipper_dict[i]) > 0:
                if len(zipper_dict[i]) == len(zipper_dict[i + 1]):

                    new_i = np.array(zipper_dict[i]).reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)
                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) > len(zipper_dict[i + 1]):

                    diff_len = len(np.array(zipper_dict[i])) - len(np.array(zipper_dict[i + 1]))
                    old_i = np.pad(zipper_dict[i + 1], (0, diff_len), 'constant')

                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) < len(zipper_dict[i + 1]):
                    # print("######less than i plus 1#########")

                    diff_len = len(zipper_dict[i + 1]) - len(zipper_dict[i])
                    old_i = np.pad(zipper_dict[i], (0, diff_len), 'constant')
                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)


        # calculate softmax function
        def softmax(s_et):
            e_set = np.exp(s_et - np.max(s_et))
            return e_set / e_set.sum(axis=0)


        for a in layer_cosine_list:
            layer_final_cos_list.append(a)
        # print(layer_final_cos_list)
        layer_softmax_values = softmax(np.ravel(np.array(layer_cosine_list, dtype=np.float32).reshape(1, -1)))
        jdk.update({layer_index: layer_softmax_values})

# jdk gives a dictionary made up of probability distributions
# print(jdk)
# print("the other one is")
b = [x for y, x in enumerate(jdk) if y != i]
all_items = list(combinations(b, 2))

# put the layers in a dictionary and rank them
other = []
#check timer here
# start the timer
start_hellinger = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()
        
for a in all_items:
    if len(jdk[a[0]]) > len(jdk[a[1]]):
        
        dif_1 = len(np.array(jdk[a[0]])) - len(np.array(jdk[a[1]]))
        new_jdk = np.pad(np.array(jdk[a[1]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[0]]).reshape(1, -1)
        #print("########################other###############") 
        new_hellinger = hellinger_fast(new_jdk[0],new_i_jdk[0])
        print(new_hellinger)
        #print("##########################end other#########")
        
        #hellinger_fast(new_i_jdk, new_jdk)
        ##kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        ##########m_no = [a[0], a[1], hellinger_fast]
        ##########other.append(m_no)

    elif len(jdk[a[0]]) < len(jdk[a[1]]):
        #check timer here
        dif_1 = len(np.array(jdk[a[1]])) - len(np.array(jdk[a[0]]))
        new_jdk = np.pad(np.array(jdk[a[0]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[1]]).reshape(1, -1)
        #print("########################other###############")        
        new_hellinger = hellinger_fast(new_jdk[0],new_i_jdk[0])
        print(new_hellinger)
        #print("##########################end other#########")       
        

        #kl_pq = rel_entr(np.ravel(new_jdk), np.ravel(new_i_jdk))
        #print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats' % sum(kl_pq))
        #m_no = [a[0], a[1], sum(kl_pq)]
        ###########hellinger_fast(new_i_jdk, new_jdk)
        ##kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        ###########m_no = [a[0], a[1], hellinger_fast]
        ##############other.append(m_no)
stop_hellinger = timeit.default_timer()
execution_time = stop_hellinger - start_hellinger
print("Hellinger execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()

KDL -selected points execution time:- 4.069349669967778
used memory 12262




















































































































































































































































































































































































































































































































































































































































































1.5949398169584326
1.5949398169584326
1.7628434338708012
1.7628434338708012
1.805955570879398
1.805955570879398
1.83898930631862
1.83898930631862
1.8586339160526508
1.8586339160526508
1.8725782992387634
1.8725782992387634
1.885350275306632
1.885350275306632
1.893283

1.961674645466147
1.961674645466147
1.9623182026629682
1.9623182026629682
1.9626253165189325
1.9626253165189325
1.963290377222418
1.963290377222418
1.9635819969803057
1.9635819969803057
1.9642720400626084
1.9642720400626084
1.96453759034018
1.96453759034018
1.9652262752625291
1.9652262752625291
1.9654639667505167
1.9654639667505167
1.9661526818819348
1.9661526818819348
1.9663754951612191
1.9663754951612191
1.9670738806144008
1.9670738806144008
1.9672739885707895
1.9672739885707895
1.9679642842380196
1.9679642842380196
1.9681607092245097
1.9681607092245097
1.968848596465278
1.968848596465278
1.969025704891574
1.969025704891574
1.9696940891491692
1.9696940891491692
1.969853803425742
1.969853803425742
1.9705154752915897
1.9705154752915897
1.9706664640767708
1.9706664640767708
1.9713030438847992
1.9713030438847992
1.9714403837062682
1.9714403837062682
1.9720711972715164
1.9720711972715164
1.9722003256716216
1.9722003256716216
1.972816381125926
1.972816381125926
1.9729327886471997
1.9735169

1.9810000989443513
1.9813522915425525
1.9813522915425525
1.9813881637943596
1.9813881637943596
1.9817307658750498
1.9817307658750498
1.9817651425236893
1.9817651425236893
1.9820985076175262
1.9820985076175262
1.982130565022283
1.982130565022283
1.9824514987050517
1.9824514987050517
1.9824810626635414
1.9827914025582403
1.9827914025582403
1.982940475084717
1.982940475084717
1.9829684510158203
1.9829684510158203
1.983120733762892
1.983120733762892
1.983147029730922
1.983147029730922
1.9833006392779293
1.9833006392779293
1.9833250540599225
1.9833250540599225
1.9834805632042312
1.9834805632042312
1.983504932264344
1.983504932264344
1.983663048886333
1.983663048886333
1.9836874661991977
1.9836874661991977
1.9838487471878252
1.9838487471878252
1.9838730370819737
1.9838730370819737
1.9840323760530383
1.9840323760530383
1.9840543136498299
1.9840543136498299
1.9842145544011873
1.9842145544011873
1.9842349924361478
1.9842349924361478
1.98439602856364
1.98439602856364
1.984415384473848
1.98441538

1.9354139605259821
1.9354139605259821
1.9354610369930294
1.9354610369930294
1.9361660461617707
1.9361660461617707
1.9362129053599717
1.9362129053599717
1.9369059677384186
1.9369059677384186
1.9369529068586768
1.9369529068586768
1.9376365646254432
1.9376365646254432
1.9376812792339873
1.9376812792339873
1.9383574119651663
1.9383574119651663
1.9384002158941693
1.9384002158941693
1.9390630121709012
1.9390630121709012
1.9391023020429026
1.9391023020429026
1.939759009923013
1.939759009923013
1.9397971340802875
1.9397971340802875
1.9404406970309265
1.9404406970309265
1.9404796983154793
1.9404796983154793
1.9411207930915122
1.9411207930915122
1.941158956366873
0.8290309127477531
0.8290309127477531
1.0418983211257342
1.0418983211257342
1.205003801054606
1.205003801054606
1.3019997027637191
1.3019997027637191
1.3708504893006357
1.3708504893006357
1.4339127093291109
1.4339127093291109
1.4730814875158313
1.4730814875158313
1.5184156866819978
1.5184156866819978
1.5433790567301275
1.543379056730127

1.7298346146280899
1.7298346146280899
1.7314908504158024
1.7314908504158024
1.7372925270386128
1.7372925270386128
1.7387862400529546
1.7387862400529546
1.7444226774378238
1.7444226774378238
1.7457691694189847
1.7457691694189847
1.7513492030765248
1.7513492030765248
1.7526234822486444
1.7526234822486444
1.75799147757091
1.75799147757091
1.7591496797106587
1.7591496797106587
1.7644693803714988
1.7644693803714988
1.7655587408886382
1.7655587408886382
1.7707536840615588
1.7707536840615588
1.771735255129661
1.7766609639064277
1.7766609639064277
1.7775332080464834
1.7775332080464834
1.7822862952383953
1.7822862952383953
1.7831488788212404
1.7831488788212404
1.7877549584091663
1.7877549584091663
1.7885203894193786
1.7885203894193786
1.7930375424127338
1.7930375424127338
1.7937632401521897
1.7937632401521897
1.798092889260469
1.798092889260469
1.7987481278030122
1.7987481278030122
1.8029825670984871
1.8029825670984871
1.8036050417983547
1.8036050417983547
1.807709724046008
1.807709724046008
1.

...

0.31992561682232523
0.31992561682232523
0.32102656897485027
0.32102656897485027
0.33910921481031153
0.33910921481031153
0.34018700278094033
0.02160016279747145
0.02160016279747145
0.023399768925904178
0.023399768925904178
0.04477622305694398
0.04477622305694398
0.046458091601136745
0.046458091601136745
0.06781116368556898
0.06781116368556898
0.06955158108960889
0.06955158108960889
0.09049398337624237
0.09049398337624237
0.09208851047664468
0.09208851047664468
0.11284019692125134
0.11284019692125134
0.11437633144702865
0.11437633144702865
0.1348815748193728
0.1348815748193728
0.13634398508821374
0.13634398508821374
0.15665306947644295
0.15665306947644295
0.15807053260361048
0.15807053260361048
0.1781288404219763
0.1781288404219763
0.17945626024021216
0.17945626024021216
0.19934397089021452
0.19934397089021452
0.20066403726893967
0.20066403726893967
0.22021656377287624
0.22021656377287624
0.22153930426605284
0.22153930426605284
0.24082408586014217
0.24082408586014217
0.2420863867983896
0

0.28145052193397296
0.28145052193397296
0.28257668731982793
0.28257668731982793
0.30107343810875553
0.30107343810875553
0.3021759092712932
0.02186087758705149
0.02186087758705149
0.0236426847203364
0.0236426847203364
0.04508312587523144
0.04508312587523144
0.04671557448374953
0.04671557448374953
0.06796076441200177
0.06796076441200177
0.069533428882704
0.069533428882704
0.09052631460559589
0.09052631460559589
0.09202350209757842
0.09202350209757842
0.1128155664211577
0.1128155664211577
0.11426673706997255
0.11426673706997255
0.1348020614285927
0.1348020614285927
0.13616104880971408
0.13616104880971408
0.15652171750184043
0.15652171750184043
0.1578731753353827
0.1578731753353827
0.17789069039911554
0.17789069039911554
0.17924488633681157
0.17924488633681157
0.19898828700582816
0.19898828700582816
0.20028060812985904
0.20028060812985904
0.21980743715870704
0.21980743715870704
0.22104258694187412
0.22104258694187412
0.2401833033234615
0.2401833033234615
0.24131890514602589
0.2413189051460

0.22128847415269598
0.22128847415269598
0.24030468692326332
0.24030468692326332
0.24145781606883004
0.24145781606883004
0.2603974185399527
0.2603974185399527
0.26152628635427144
0.0217532981362388
0.0217532981362388
0.023363573353013967
0.023363573353013967
0.0448585326467625
0.0448585326467625
0.046391526560008285
0.046391526560008285
0.06768086412155036
0.06768086412155036
0.06916673968866971
0.06916673968866971
0.09019319751198179
0.09019319751198179
0.09158468660575805
0.09158468660575805
0.11243230996914129
0.11243230996914129
0.1138160882897063
0.1138160882897063
0.13431235270467315
0.13431235270467315
0.13569893499919172
0.13569893499919172
0.15591452689744997
0.15591452689744997
0.15723775664393655
0.15723775664393655
0.17723159804162367
0.17723159804162367
0.17849628727400846
0.17849628727400846
0.19809478079027046
0.19809478079027046
0.19925754292799952
0.19925754292799952
0.21867532306338247
0.21867532306338247
0.21980191628879447
0.21980191628879447
0.23883402128669298
0.23

0.11212380829142332
0.11212380829142332
0.11347846129173338
0.11347846129173338
0.13394709040713226
0.13394709040713226
0.1352418109738493
0.1352418109738493
0.15530570325695262
0.15530570325695262
0.1564960779210512
0.1564960779210512
0.17637496494264526
0.17637496494264526
0.17752831036560912
0.17752831036560912
0.19701236628811716
0.19701236628811716
0.1981938657374978
0.1981938657374978
0.2175994233681866
0.2175994233681866
0.21875606499356784
0.0015379168439573518
0.0015379168439573518
0.023300845722049327
0.023300845722049327
0.024741072067689343
0.024741072067689343
0.04631890045216547
0.04631890045216547
0.04775114440925597
0.04775114440925597
0.06896530951891164
0.06896530951891164
0.07040045616313145
0.07040045616313145
0.09132411473512805
0.09132411473512805
0.09269369299008824
0.09269369299008824
0.11338783501870267
0.11338783501870267
0.11469681976242473
0.11469681976242473
0.13498176530788636
0.13498176530788636
0.1361852552507084
0.1361852552507084
0.15628315710350082
0.

0.024575821997402488
0.024575821997402488
0.04600903877607607
0.04600903877607607
0.04736477149921603
0.04736477149921603
0.06837417632319769
0.06837417632319769
0.06962064909410978
0.06962064909410978
0.09043632929100866
0.09043632929100866
0.09164402503985196
0.09164402503985196
0.11204627256863581
0.11204627256863581
0.11328345172148865
0.11328345172148865
0.1336034956463468
0.1336034956463468
0.13481464629614437
0.001486399944989186
0.001486399944989186
0.02315732924521271
0.02315732924521271
0.024575821997402488
0.024575821997402488
0.04600903877607607
0.04600903877607607
0.04736477149921603
0.04736477149921603
0.06837417632319769
0.06837417632319769
0.06962064909410978
0.06962064909410978
0.09043632929100866
0.09043632929100866
0.09164402503985196
0.09164402503985196
0.11204627256863581
0.11204627256863581
0.11328345172148865
0.11328345172148865
0.1336034956463468
0.1336034956463468
0.13481464629614437
0.02168704763468985
0.02168704763468985
0.02310659546906342
0.0231065954690634